In [ ]:
import tweepy
from textblob import TextBlob
import pandas as pd
from plotly import __version__
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()
import matplotlib.pyplot as plt
from tweepy import Stream,StreamListener
import json,re,csv


In [ ]:
consumer_key = 'xEMZLfBL4XpKv4Qsk0MPEp7tj'
consumer_key_secret = 'JW4OCCd7K3Lzn0yX9IH3Wcefj9XvFHfqhY5SG7DoVgYv1Fp0BZ'

access_token = '445449105-cD38nd3sgPsVyZ9y92yyc2jm6cpNaV0lCvo3F75H'
access_token_secret = 'hNkq3yP7Qr2OnFZ0WW71qh86mcee6A9C4DHar4MhDbM88'

auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)

auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
pos1, neg1, neu1 = 0, 0, 0
header=['Positive','Negative','Neutral']

#Create an empty csv file which has three headers: 'Positive','Negative','Neutral'
with open('List.csv','w') as file:
    write=csv.DictWriter(file,fieldnames=header)
    write.writeheader()
    
#doc for Stream Listener to tweepy: http://docs.tweepy.org/en/latest/streaming_how_to.html
class Listener(StreamListener):
    
    def on_data(self, data):
        raw_t=json.loads(data)
        
        try:
            #this assumes that the data in json has a 'text' field, if not it will go to except and print the message 
            data=raw_t['text']
            #four lines below will clear the tweets by removing: metions, has tag etc.
            data = re.sub('@[A-Za-z0–9]+', '',data) #Removing @mentions
            data = re.sub('#', '', data) # Removing '#' hash tag
            data = re.sub('RT[\s]+', '', data) # Removing RT
            data = re.sub('https?:\/\/\S+', '', data) # Removing hyperlink


            global pos1 
            global neg1 
            global neu1
            pos, neg, neu = 0, 0, 0


            print(data)
            analysis = TextBlob(data)
            print(analysis.sentiment)
            
            #the below if statement will count the number of tweets based on their sentiment('Positive','Negative','Neutral')
            if analysis.sentiment[0]>0:
                pos+=1
            elif analysis.sentiment[0]<0:
                neg+=1
            else:
                neu+=1
            pos1=pos1+pos
            neg1=neg1+neg
            neu1=neu1+neu
            
            #write the result from counting to the csv file "List.csv"
            with open('List.csv', 'a') as file:
                writer = csv.DictWriter(file, fieldnames=header)
                info={
                    'Positive':pos1,
                    'Negative':neg1,
                    'Neutral':neu1
                      }
                writer.writerow(info)

            print(data)

            return True
        except:
            print('You did something wrong, take a look at the data')
        

    def on_error(self, status):
        print(status)


l = Listener()

In [ ]:
stream = Stream(auth, l)
stream.filter(track=['bloomberg'])
